# Import libraries

In [1]:
import os
import warnings
from concurrent.futures import ThreadPoolExecutor

import numpy as np
import pandas as pd

from IPython.display import clear_output

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score
from sklearn.base import clone
from sklearn.impute import KNNImputer
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.optimize import minimize

import torch

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor


from tqdm import tqdm
from colorama import Fore, Style

warnings.filterwarnings('ignore')


In [2]:
import random

SEED = 42
np.random.seed(SEED)
random.seed(SEED)
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
seed_everything(2024)

# Feature engineering helper functions

## Time series

In [3]:
def dictionary_of_statistics(data, time = None):
    try:
        #Handle empty dataframe
        if (data.empty or len(data.columns) == 0 or data is None):
            return {}
        
        if len(data.columns) == 0:
            return {}
        
        #Aggreate statistics for the dictionary
        stats_summary = data.agg(['mean', 'median', 'max', 'std']).to_dict()
        
        flattened_stats = {}
        for col, stats in stats_summary.items():
            for stat_name, value in stats.items():
                key = f"{stat_name}_{col}"
                if (time is not None):
                    key = f"{stat_name}_{col}_{time}"
                flattened_stats[key] = value
        
        return flattened_stats
    except Exception as e:
        return {}

#Feature engineering

def compute_time_features(data, day_start_hour=6, day_end_hour=18, expected_diff=5):
    
    """
    Compute and add time-related features to the DataFrame.

    Parameters:
    - data (pd.DataFrame): The input DataFrame containing 'time_of_day' in nanosecond and 'relative_date_PCIAT'.
    - day_start_hour (int): Hour to start the day period. Default is 8.
    - day_end_hour (int): Hour to end the day period. Default is 21.
    - expected_diff (int): Expected time difference between steps in seconds. Default is 5.
    
    """
    
    #From nanosecond to hour in a day
    data['time_of_day_hours'] = data['time_of_day'] / 1e9 / 3600
    data['day_time'] = data['relative_date_PCIAT'] + data['time_of_day_hours'] / 24
    
    #Categorize the day and night based on time data
    
    data['day_period'] = np.where(
        (data['time_of_day_hours'] >= day_start_hour) &
        (data['time_of_day_hours'] < day_end_hour),
        'day', 'night'
    )
    
    #Time difference beween steps
    #As the description, the time_of_day should represent the start of a 5s window over which the data was sampled
    #Calculate the time difference between each step
    data['time_diff'] = (data['day_time'].diff() * 86400).round(0) # seconds in a day
    data['measurement_after_gap'] = data['time_diff'] > expected_diff
    
def no_motion_periods(worn_data):
    """
    Find periods of no motion and give analytical insights in the data.

    Parameters:
    - data (pd.DataFrame): The input DataFrame containing 'time_of_day' and 'relative_date_PCIAT'.

    Returns:
    - pd.DataFrame: DataFrame with new features: 
    + total duration of no motion periods per day
    + the number of no motion periods per day.
    """
    
    #Calculate no motion periods
    no_motion = worn_data['enmo'] == 0
    motion_group = (
        (no_motion != no_motion.shift()) |
        (worn_data['measurement_after_gap'])
    ).cumsum()

    no_motion_periods = worn_data[no_motion].groupby(
        motion_group
    )['day_time'].agg(['min', 'max'])

    no_motion_periods['duration_sec'] = (
        (no_motion_periods['max'] - no_motion_periods['min']) * 86400
    ).round(0).astype(int)
    
    no_motion_periods['duration_sec'] += 5
    no_motion_periods['day'] = no_motion_periods['min'].astype(int)
    
    # Calculate daily statistics on no motion periods
    daily_stats = no_motion_periods.groupby(no_motion_periods['day']) \
        .agg(no_motion_duration=('duration_sec', 'sum'),
            no_motion_count=('duration_sec', 'size'))
    
    #Aggreate statistics for the dictionary
    return dictionary_of_statistics(daily_stats)

def circadian_rhythm_analysis(worn_data):
    
    """
    Make features capturing the variation in activity across the 24-hour cycle, 
    separately for day and night times (or wakefulness and sleep periods).
    
    Parameters:
    - data (pd.DataFrame): The input DataFrame containing 'time_of_day_hours', 'day_period' and 'relative_date_PCIAT'.
    
    Returns:
    - pd.DataFrame: 2 DataFrame, corresponding to day and night times,  with new features capturing the circadian rhythm of the wearer,
    + Standard deviation across hourly means per day
    + Peak hour of activity per day
    + Entropy of activity distribution per day
    """
    
    try:
        if (worn_data.empty or worn_data is None):
            return {}
    
        hourly_activity = worn_data.groupby(
            [worn_data['relative_date_PCIAT'].astype(int),
            worn_data['time_of_day_hours'].astype(int),
            worn_data['day_period']]
        )['enmo'].agg(['mean', 'max'])

        features = hourly_activity['mean'].groupby(
            ['relative_date_PCIAT', 'day_period']
        ).agg(
            std_across_hours='std',
            peak_hour=lambda x: x.idxmax()[1],
            entropy=lambda x: -(x / x.sum() * np.log(x / x.sum() + 1e-9)).sum()
        )
        
        # Safely extract day/night features
        try:
            day_features = features.xs('day', level='day_period')
        except:
            day_features = pd.DataFrame()
            
        try:    
            night_features = features.xs('night', level='day_period')
        except:
            night_features = pd.DataFrame()

        return dictionary_of_statistics(day_features, time="day") | dictionary_of_statistics(night_features, time='night')  
    except Exception as e:
        return {}

def physical_activity_analysis(worn_data):
        
    """
    Analyze the Moderate to Vigorous Physical Activity (MVPA) based on a threshold of ENMO values, 
    and calculate the duration of the detected MVPA activity bouts
    
    Parameters:
    - data (pd.DataFrame): The input DataFrame containing 'enmo', 'time_diff' and 'day_time'.
    
    Returns:
    - pd.DataFrame: DataFrame with new features capturing the physical activity level of the wearer,
    including:
    + Total duration of MVPA per day
    + Number of MVPA periods per day
    """
    # In order to classify physical activity as MVPA, we only retained activities that lasted at least 1 minute and met the criteria for the 100 mg (= 0.1g) threshold
    if worn_data is None or worn_data.empty:
        return {}
    
    mvpa_threshold = 0.1
    merge_gap = 60
    
    def merge_mvpa_groups(df, allowed_gap=60, merge_gap=60):
        last_mvpa_time = df['day_time'].where(df['is_mvpa']).ffill().shift()
        
        mvpa_time_diff = (
            (df['day_time'] - last_mvpa_time) * 86400
        ).round(0)
        
        mvpa_group = (
            (df['is_mvpa'] != df['is_mvpa'].shift()) |
            (df['time_diff'] >= allowed_gap)
        ).cumsum()
        
        is_mvpa_start = (
            (mvpa_group != mvpa_group.shift()) &
            df['is_mvpa']
        )
        
        group_increment = is_mvpa_start & (
            (mvpa_time_diff >= merge_gap) | last_mvpa_time.isnull()
        )
        
        merged_group = group_increment.cumsum()
        merged_group.loc[~df['is_mvpa']] = np.nan
        
        return merged_group
    
    worn_data['is_mvpa'] = worn_data['enmo'] > mvpa_threshold
    worn_data['mvpa_merged_group'] = merge_mvpa_groups(worn_data)

    mvpa_periods = worn_data[
        worn_data['is_mvpa']
    ].groupby('mvpa_merged_group')['day_time'].agg(['min', 'max'])

    mvpa_periods['duration_sec'] = (
        mvpa_periods['max'] - mvpa_periods['min']
    ) * 86400  # days to seconds

    mvpa_periods = mvpa_periods[mvpa_periods['duration_sec'] >= 60]
    mvpa_periods['duration_min'] = mvpa_periods['duration_sec'] / 60
    
    mvpa_periods['day'] = mvpa_periods['min'].astype(int)

    daily_stats = mvpa_periods.groupby(mvpa_periods['day']) \
        .agg(mvpa_total_duration=('duration_sec', 'sum'),
            mvpa_count_periods=('duration_sec', 'size'))
        
    return dictionary_of_statistics(daily_stats)
    
def activity_transition_analysis(worn_data):
        
    """
    The analysis to look at transitions between low, moderate and vigorous activity. To smooth out sudden, 
    short bursts of different activities, we filter out segments with a duration below a 1 minute threshold.
    
    Parameters:
    - data (pd.DataFrame): The input DataFrame containing 'enmo', 'time_diff' and 'day_time'.
    
    Returns:
    - pd.DataFrame: DataFrame with new features capturing the activity transitions of the wearer,
    + Total duration of different of activity per day
    + Number of different activity periods per day
    """
    mvpa_threshold = 0.1
    vig_threshold = 0.5
    worn_data['activity_type'] = pd.cut(
        worn_data['enmo'],
        bins=[-np.inf, mvpa_threshold, vig_threshold, np.inf],
        labels=['low', 'moderate', 'vigorous']
    )
    activity_group = (
        (worn_data['activity_type'] != worn_data['activity_type'].shift()) |
        (worn_data['measurement_after_gap'])
    ).cumsum()
    
    activity_periods = worn_data.groupby(activity_group).agg(
        min=('day_time', 'min'),
        max=('day_time', 'max'),
        activity_type=('activity_type', 'first')
    )
    activity_periods['duration_sec'] = (
        activity_periods['max'] - activity_periods['min']
    ) * 86400 + 5 

    activity_periods = activity_periods[activity_periods['duration_sec'] >= 60]
    activity_periods['duration_min'] = activity_periods['duration_sec'] / 60
    
    activity_periods['day'] = activity_periods['min'].astype(int)
    activity_periods['transition_num'] = (
        activity_periods.groupby('day')['activity_type']
        .apply(lambda x: (x != x.shift()).cumsum())
        .reset_index(level=0, drop=True)
    )
    
    low_activity = activity_periods[activity_periods['activity_type'] == 'low'].groupby('day').agg(
        low_act_total_duration=('duration_sec', 'sum'),
        low_act_count_periods=('duration_sec', 'size')
    )
    
    moderate_activity = activity_periods[activity_periods['activity_type'] == 'moderate'].groupby('day').agg(
            moderate_act_total_duration=('duration_sec', 'sum'),
            moderate_act_count_periods=('duration_sec', 'size')
    )

        
    daily_transitions = activity_periods.groupby('day').agg(
        count_transitions=('transition_num', 'max')
    )
    
    return dictionary_of_statistics(low_activity) | dictionary_of_statistics(moderate_activity) | dictionary_of_statistics(daily_transitions)

def activity_light_exposure(worn_data):
        
    """
    Analyze the correlation between light exposure and physical activity level.
    
    Parameters:
    - data (pd.DataFrame): The input DataFrame containing 'light' and 'enmo'.
    
    Returns:
    - float: The correlation between light exposure and physical activity level.
    """
    correlation_light_enmo = worn_data[['light', 'enmo']].corr().iloc[0, 1]
    return {'correlation_light_enmo': correlation_light_enmo}

def process_file(file_path, participant_id):
    try:
        data = pd.read_parquet(file_path)
        
        if data.empty:
            return {'id': participant_id}
        
        required_columns = ['time_of_day', 'relative_date_PCIAT', 'enmo', 'non-wear_flag', 'light']
        if not all(col in data.columns for col in required_columns):
            return {'id': participant_id}

        # Compute time features
        compute_time_features(data)
            
        # Calculate the percentage of non-worn time
        non_wear_percentage = (data['non-wear_flag'].sum() / len(data)) * 100
        
        # Filter out the worn data
        worn_data = data[data['non-wear_flag'] == 0]
        
        if (worn_data.empty):
            return {'id': participant_id}
        
        # recalculate time difference between rows and measurement_after_gap flag in the worn data
        expected_diff = 5
        worn_data['time_diff'] = (worn_data['day_time'].diff() * 86400).round(0)
        worn_data['measurement_after_gap'] = worn_data['time_diff'] > expected_diff

        # Compute no motion periods
        no_motion_stats = no_motion_periods(worn_data)
        # Circadian rhythm analysis
        circadian_rhythm_stats = circadian_rhythm_analysis(worn_data)
        # Physical activity analysis
        physical_activity_stats = physical_activity_analysis(worn_data)
        # Activity transition analysis
        activity_transition_stats = activity_transition_analysis(worn_data)
        # Compute activity light correlation
        activity_light_stats = activity_light_exposure(worn_data)

        return {
            'id': participant_id,
        } | no_motion_stats | circadian_rhythm_stats | physical_activity_stats | activity_transition_stats | activity_light_stats
    
    except Exception as e:
        return {'id': participant_id}

def load_time_series(dir_name):
    
    participant_ids = os.listdir(dir_name)
    
    with ThreadPoolExecutor() as executor:
        #tqdm: Wraps the executor.map iterable with tqdm -> Show a progress bar indicating the processing status
        results = list(tqdm(executor.map(lambda x: process_file(os.path.join(dir_name, x, 'part-0.parquet'), x), participant_ids), total=len(participant_ids)))

    df = pd.DataFrame(results)
    # Replace inf and -inf with NaN
    df.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Replace NaN with 0
    df.fillna(0, inplace=True)
    
    return df

## Tabular

In [4]:
def feature_engineering(df):
    df['BMI_Age'] = df['Physical-BMI'] * df['Basic_Demos-Age']
    df['Internet_Hours_Age'] = df['PreInt_EduHx-computerinternet_hoursday'] * df['Basic_Demos-Age']
    df['BMI_Internet_Hours'] = df['Physical-BMI'] * df['PreInt_EduHx-computerinternet_hoursday']
    df['BFP_BMI'] = df['BIA-BIA_Fat'] / df['BIA-BIA_BMI']
    df['FFMI_BFP'] = df['BIA-BIA_FFMI'] / df['BIA-BIA_Fat']
    df['FMI_BFP'] = df['BIA-BIA_FMI'] / df['BIA-BIA_Fat']
    df['LST_TBW'] = df['BIA-BIA_LST'] / df['BIA-BIA_TBW']
    df['BFP_BMR'] = df['BIA-BIA_Fat'] * df['BIA-BIA_BMR']
    df['BFP_DEE'] = df['BIA-BIA_Fat'] * df['BIA-BIA_DEE']
    df['BMR_Weight'] = df['BIA-BIA_BMR'] / df['Physical-Weight']
    df['DEE_Weight'] = df['BIA-BIA_DEE'] / df['Physical-Weight']
    df['SMM_Height'] = df['BIA-BIA_SMM'] / df['Physical-Height']
    df['Muscle_to_Fat'] = df['BIA-BIA_SMM'] / df['BIA-BIA_FMI']
    df['Hydration_Status'] = df['BIA-BIA_TBW'] / df['Physical-Weight']
    df['ICW_TBW'] = df['BIA-BIA_ICW'] / df['BIA-BIA_TBW']
    df['BMI_PHR'] = df['Physical-BMI'] * df['Physical-HeartRate']

    # Replace any remaining inf values with NaN
    df = df.replace([np.inf, -np.inf], np.nan)
    
    return df

## Tabular KNN Imputer

In [5]:
def numeric_imputation(train_df, test_df, dif_cols):
    numeric_cols = train_df.select_dtypes(include=['float64', 'int64', 'int32', 'float32']).columns.tolist()
    
    numeric_cols = [col for col in numeric_cols if col not in dif_cols]

    imputer = KNNImputer(n_neighbors=5)

    #Fit on training set
    imputer.fit(train_df[numeric_cols])

    #Transform both
    train_df[numeric_cols] = imputer.transform(train_df[numeric_cols])
    test_df[numeric_cols] = imputer.transform(test_df[numeric_cols])

    return train_df, test_df

# Data loading

## Load time series data

In [6]:
train_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet")
test_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet")

100%|██████████| 2/2 [00:00<00:00,  5.82it/s]


In [7]:
train_ts['id'] = train_ts['id'].str.replace('id=', '')
test_ts['id'] = test_ts['id'].str.replace('id=', '')

## Load tablular data

In [8]:
train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
sample = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv')

In [9]:
train_cols = set(train.columns)
test_cols = set(test.columns)
dif_cols = sorted(list(train_cols - test_cols))

print(dif_cols)

['PCIAT-PCIAT_01', 'PCIAT-PCIAT_02', 'PCIAT-PCIAT_03', 'PCIAT-PCIAT_04', 'PCIAT-PCIAT_05', 'PCIAT-PCIAT_06', 'PCIAT-PCIAT_07', 'PCIAT-PCIAT_08', 'PCIAT-PCIAT_09', 'PCIAT-PCIAT_10', 'PCIAT-PCIAT_11', 'PCIAT-PCIAT_12', 'PCIAT-PCIAT_13', 'PCIAT-PCIAT_14', 'PCIAT-PCIAT_15', 'PCIAT-PCIAT_16', 'PCIAT-PCIAT_17', 'PCIAT-PCIAT_18', 'PCIAT-PCIAT_19', 'PCIAT-PCIAT_20', 'PCIAT-PCIAT_Total', 'PCIAT-Season', 'sii']


## Fill sii with possible maximized PCIAT values

In [10]:
PCIAT_cols = [f'PCIAT-PCIAT_{i+1:02d}' for i in range(20)]

#Recalculates the SII value based on the the current PCIAT values and the possible maximum PCIAT values
def recalculate_sii(row):
    value = 0
    if (not pd.isna(row['PCIAT-PCIAT_Total'])):
        value = row['PCIAT-PCIAT_Total']
        
    max_possible = value + row[PCIAT_cols].isna().sum() * 5
    
    if value <= 30 and max_possible <= 30:
        return 0
    elif 31 <= value <= 49 and max_possible <= 49:
        return 1
    elif 50 <= value <= 79 and max_possible <= 79:
        return 2
    elif value >= 80 and max_possible >= 80:
        return 3
    
    return np.nan

train['recalc_sii'] = train.apply(recalculate_sii, axis=1)
train['sii'] = train['recalc_sii']
train.drop(columns='recalc_sii', inplace=True)

## Data merging

In [11]:
selected_features = ['Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
                'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
                'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
                'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
                'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage',
                'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
                'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
                'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
                'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
                'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
                'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
                'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
                'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
                'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
                'BIA-BIA_TBW', 'PAQ_A-Season', 'PAQ_A-PAQ_A_Total', 'PAQ_C-Season',
                'PAQ_C-PAQ_C_Total', 'SDS-Season', 'SDS-SDS_Total_Raw',
                'SDS-SDS_Total_T', 'PreInt_EduHx-Season',
                'PreInt_EduHx-computerinternet_hoursday', 'sii']

cat_c = ['Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 
          'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season', 
          'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season']

# Preprocessed time series

time_series_cols = train_ts.columns.tolist()
time_series_cols.remove("id")

selected_features += time_series_cols

train = pd.merge(train, train_ts, how="left", on='id')
test = pd.merge(test, test_ts, how="left", on='id')

train = train.drop('id', axis=1)
test = test.drop('id', axis=1)   

train = train[selected_features]

# Drop n/a values
train = train.dropna(subset='sii')

def update(df):
    global cat_c
    for c in cat_c: 
        df[c] = df[c].fillna('Missing')
        df[c] = df[c].astype('category')
    return df
        
train = update(train)
test = update(test)

## Handle missing values in training set

In [12]:
train, test = numeric_imputation(train, test, dif_cols)
train = feature_engineering(train)

train = train.dropna(thresh=10, axis=0)
test = feature_engineering(test)


def create_mapping(column, dataset):
    unique_values = dataset[column].unique()
    return {value: idx for idx, value in enumerate(unique_values)}

for col in cat_c:
    mapping = create_mapping(col, train)
    mappingTe = create_mapping(col, test)
    
    train[col] = train[col].replace(mapping).astype(int)
    test[col] = test[col].replace(mappingTe).astype(int)

# Supervised learning phase

## Voting regressors with parameters tuning

In [13]:
lightgbm_param_grid = {
    'learning_rate': [0.04, 0.046, 0.05],
    'max_depth': [10, 12, 14],
    'num_leaves': [450, 478, 500],
    'min_data_in_leaf': [12, 13, 14],
    'feature_fraction': [0.88, 0.893, 0.9],
    'bagging_fraction': [0.78, 0.784, 0.79],
    'bagging_freq': [3, 4, 5],
    'lambda_l1': [8, 10, 12],
    'lambda_l2': [0.005, 0.01, 0.02],
    'n_estimators': [200, 250, 300]
}


xgboost_param_grid = {
    'learning_rate': [0.045, 0.05, 0.055],
    'max_depth': [5, 6, 7],
    'n_estimators': [150, 200, 250],
    'subsample': [0.75, 0.8, 0.85],
    'colsample_bytree': [0.75, 0.8, 0.85],
    'reg_alpha': [0.8, 1, 1.2],
    'reg_lambda': [4, 5, 6],
    'random_state': [SEED]
}


catboost_param_grid = {
    'learning_rate': [0.045, 0.05, 0.055],  # Around 0.05
    'depth': [5, 6, 7],  # Around 6
    'iterations': [150, 200, 250],  # Around 200
    'l2_leaf_reg': [8, 10, 12],  # Around 10
    'random_seed': [SEED],
    'verbose': [0]
}


lgbm_model = LGBMRegressor(verbosity=-1)
xgb_model = XGBRegressor(verbosity=0)
catboost_model = CatBoostRegressor(cat_features=cat_c, verbose=0)

lgbm_random = RandomizedSearchCV(estimator=lgbm_model, param_distributions=lightgbm_param_grid, 
                                  scoring='accuracy', cv=3, n_iter=10, verbose=2, random_state=SEED)

xgb_random = RandomizedSearchCV(estimator=xgb_model, param_distributions=xgboost_param_grid, 
                                 scoring='accuracy', cv=3, n_iter=10, verbose=2, random_state=SEED)

catboost_random = RandomizedSearchCV(estimator=catboost_model, param_distributions=catboost_param_grid, 
                                      scoring='accuracy', cv=3, n_iter=10, verbose=2, random_state=SEED)


x_train_pd = train[[col for col in train.columns if col != 'sii']]
y_train_pd = train['sii']


lgbm_random.fit(x_train_pd, y_train_pd)
xgb_random.fit(x_train_pd, y_train_pd)
catboost_random.fit(x_train_pd, y_train_pd)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END bagging_fraction=0.79, bagging_freq=5, feature_fraction=0.893, lambda_l1=12, lambda_l2=0.01, learning_rate=0.04, max_depth=12, min_data_in_leaf=14, n_estimators=200, num_leaves=478; total time=   0.6s
[CV] END bagging_fraction=0.79, bagging_freq=5, feature_fraction=0.893, lambda_l1=12, lambda_l2=0.01, learning_rate=0.04, max_depth=12, min_data_in_leaf=14, n_estimators=200, num_leaves=478; total time=   0.6s
[CV] END bagging_fraction=0.79, bagging_freq=5, feature_fraction=0.893, lambda_l1=12, lambda_l2=0.01, learning_rate=0.04, max_depth=12, min_data_in_leaf=14, n_estimators=200, num_leaves=478; total time=   0.6s
[CV] END bagging_fraction=0.78, bagging_freq=5, feature_fraction=0.893, lambda_l1=8, lambda_l2=0.02, learning_rate=0.04, max_depth=10, min_data_in_leaf=12, n_estimators=200, num_leaves=450; total time=   0.7s
[CV] END bagging_fraction=0.78, bagging_freq=5, feature_fraction=0.893, lambda_l1=8, lambda_l2=0.02,

RandomizedSearchCV(cv=3,
                   estimator=<catboost.core.CatBoostRegressor object at 0x7e7a82be9390>,
                   param_distributions={'depth': [5, 6, 7],
                                        'iterations': [150, 200, 250],
                                        'l2_leaf_reg': [8, 10, 12],
                                        'learning_rate': [0.045, 0.05, 0.055],
                                        'random_seed': [42], 'verbose': [0]},
                   random_state=42, scoring='accuracy', verbose=2)

In [14]:
print("Best LightGBM Params:", lgbm_random.best_params_)
print("Best LightGBM Score:", lgbm_random.best_score_)

print("Best XGBoost Params:", xgb_random.best_params_)
print("Best XGBoost Score:", xgb_random.best_score_)

print("Best CatBoost Params:", catboost_random.best_params_)
print("Best CatBoost Score:", catboost_random.best_score_)

Best LightGBM Params: {'num_leaves': 478, 'n_estimators': 200, 'min_data_in_leaf': 14, 'max_depth': 12, 'learning_rate': 0.04, 'lambda_l2': 0.01, 'lambda_l1': 12, 'feature_fraction': 0.893, 'bagging_freq': 5, 'bagging_fraction': 0.79}
Best LightGBM Score: nan
Best XGBoost Params: {'subsample': 0.85, 'reg_lambda': 5, 'reg_alpha': 1.2, 'random_state': 42, 'n_estimators': 200, 'max_depth': 6, 'learning_rate': 0.045, 'colsample_bytree': 0.8}
Best XGBoost Score: nan
Best CatBoost Params: {'verbose': 0, 'random_seed': 42, 'learning_rate': 0.045, 'l2_leaf_reg': 10, 'iterations': 150, 'depth': 6}
Best CatBoost Score: nan


## Training functions

In [15]:
n_splits=5

def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))

def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

def modelTraining(model_class, train_data, test_data):
    X = train_data.drop(['sii'], axis=1)
    y = train_data['sii']

    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    train_S = []
    test_S = []
    
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    oof_rounded = np.zeros(len(y), dtype=int) 
    test_preds = np.zeros((len(test_data), n_splits))

    for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=n_splits)):
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

        model = clone(model_class)
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)

        oof_non_rounded[test_idx] = y_val_pred
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        oof_rounded[test_idx] = y_val_pred_rounded

        train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
        val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

        train_S.append(train_kappa)
        test_S.append(val_kappa)
        
        test_preds[:, fold] = model.predict(test_data)
        
        print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
        clear_output(wait=True)

    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

    KappaOPtimizer = minimize(evaluate_predictions,
                              x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
                              method='Nelder-Mead')
    assert KappaOPtimizer.success, "Optimization did not converge."
    
    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")

    tpm = test_preds.mean(axis=1)
    tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)
    
    submission = pd.DataFrame({
        'id': sample['id'],
        'sii': tpTuned
    })

    return submission

In [16]:
Light = LGBMRegressor(**lgbm_random.best_params_, verbose=-1)
XGB_Model = XGBRegressor(**xgb_random.best_params_)
CatBoost_Model = CatBoostRegressor(**catboost_random.best_params_)

# Combine models using Voting Regressor
voting_model = VotingRegressor(estimators=[
    ('lightgbm', Light),
    ('xgboost', XGB_Model),
    ('catboost', CatBoost_Model)
], weights=[4.0, 4.0, 5.0])

## Model training and submission

In [17]:
submission = modelTraining(voting_model, train, test)
submission.to_csv('submission.csv', index=False)
print(submission['sii'].value_counts())

Training Folds: 100%|██████████| 5/5 [00:26<00:00,  5.27s/it]

Mean Train QWK --> 0.7718
Mean Validation QWK ---> 0.3837


----> || Optimized QWK SCORE ::  0.460
sii
0    9
1    6
2    5
Name: count, dtype: int64
